In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import (roc_auc_score, f1_score, classification_report,
                             confusion_matrix, roc_curve, precision_recall_curve,
                             average_precision_score)
import joblib
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


os.makedirs("../models/saved_models", exist_ok=True)
os.makedirs("../reports/figures", exist_ok=True)

def evaluate_model(model, X_train, y_train, X_test, y_test, model_name, cv_folds=5):
    """
    Comprehensive model evaluation with cross-validation and multiple metrics
    """
    print(f"\n{'='*60}")
    print(f"Evaluating {model_name}")
    print(f"{'='*60}")

    # Time tracking
    start_time = datetime.now()

    # Cross-validation
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    cv_scores_auc = cross_val_score(model, X_train, y_train,
                                   cv=cv, scoring='roc_auc', n_jobs=-1)
    cv_scores_f1 = cross_val_score(model, X_train, y_train,
                                  cv=cv, scoring='f1', n_jobs=-1)

    # Train model
    model.fit(X_train, y_train)

    # Predictions
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)

    # Calculate metrics
    train_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
    test_auc = roc_auc_score(y_test, y_pred_proba)
    test_f1 = f1_score(y_test, y_pred)
    test_ap = average_precision_score(y_test, y_pred_proba)

    # Time taken
    end_time = datetime.now()
    time_taken = (end_time - start_time).total_seconds()

    # Create results dictionary
    results = {
        'Model': model_name,
        'Train_AUC': round(train_auc, 4),
        'Test_AUC': round(test_auc, 4),
        'Test_F1': round(test_f1, 4),
        'Test_AP': round(test_ap, 4),
        'CV_AUC_Mean': round(cv_scores_auc.mean(), 4),
        'CV_AUC_Std': round(cv_scores_auc.std(), 4),
        'CV_F1_Mean': round(cv_scores_f1.mean(), 4),
        'CV_F1_Std': round(cv_scores_f1.std(), 4),
        'Time_Seconds': round(time_taken, 2)
    }

    # Print results
    print(f"Training AUC: {train_auc:.4f}")
    print(f"Test AUC: {test_auc:.4f}")
    print(f"Test F1-Score: {test_f1:.4f}")
    print(f"Average Precision: {test_ap:.4f}")
    print(f"CV AUC: {cv_scores_auc.mean():.4f} (±{cv_scores_auc.std():.4f})")
    print(f"CV F1: {cv_scores_f1.mean():.4f} (±{cv_scores_f1.std():.4f})")
    print(f"Time taken: {time_taken:.2f} seconds")

    # Generate plots
    generate_model_plots(model, X_test, y_test, y_pred, y_pred_proba, model_name)

    return results, model

def generate_model_plots(model, X_test, y_test, y_pred, y_pred_proba, model_name):
    """
    Generate comprehensive evaluation plots for each model
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'{model_name} - Model Evaluation', fontsize=16, fontweight='bold')

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0])
    axes[0, 0].set_title('Confusion Matrix')
    axes[0, 0].set_xlabel('Predicted')
    axes[0, 0].set_ylabel('Actual')

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    axes[0, 1].plot(fpr, tpr, color='darkorange', lw=2,
                   label=f'ROC curve (AUC = {roc_auc:.4f})')
    axes[0, 1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    axes[0, 1].set_xlim([0.0, 1.0])
    axes[0, 1].set_ylim([0.0, 1.05])
    axes[0, 1].set_xlabel('False Positive Rate')
    axes[0, 1].set_ylabel('True Positive Rate')
    axes[0, 1].set_title('ROC Curve')
    axes[0, 1].legend(loc="lower right")

    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    ap = average_precision_score(y_test, y_pred_proba)
    axes[1, 0].plot(recall, precision, color='green', lw=2,
                   label=f'PR curve (AP = {ap:.4f})')
    axes[1, 0].set_xlim([0.0, 1.0])
    axes[1, 0].set_ylim([0.0, 1.05])
    axes[1, 0].set_xlabel('Recall')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].set_title('Precision-Recall Curve')
    axes[1, 0].legend(loc="upper right")

    # Feature Importance (if available)
    if hasattr(model, 'feature_importances_'):
        feature_importances = pd.DataFrame({
            'feature': range(len(model.feature_importances_)),
            'importance': model.feature_importances_
        })
        feature_importances = feature_importances.sort_values('importance', ascending=False).head(10)
        axes[1, 1].barh(feature_importances['feature'], feature_importances['importance'])
        axes[1, 1].set_xlabel('Importance')
        axes[1, 1].set_ylabel('Feature Index')
        axes[1, 1].set_title('Top 10 Feature Importances')
    else:
        # For logistic regression, show coefficients
        if hasattr(model, 'coef_'):
            coef = pd.DataFrame({
                'feature': range(len(model.coef_[0])),
                'coefficient': model.coef_[0]
            })
            coef['abs_coef'] = np.abs(coef['coefficient'])
            coef = coef.sort_values('abs_coef', ascending=False).head(10)
            axes[1, 1].barh(coef['feature'], coef['coefficient'])
            axes[1, 1].set_xlabel('Coefficient Value')
            axes[1, 1].set_ylabel('Feature Index')
            axes[1, 1].set_title('Top 10 Feature Coefficients (Logistic Regression)')
        else:
            axes[1, 1].text(0.5, 0.5, 'Feature importance\nnot available',
                           ha='center', va='center', transform=axes[1, 1].transAxes)
            axes[1, 1].set_title('Feature Importance')

    plt.tight_layout()
    plt.savefig(f"../reports/figures/{model_name.replace(' ', '_').lower()}_evaluation.svg",
                format='svg', bbox_inches='tight')
    plt.close()

def plot_comparison_results(results_df):
    """
    Create comprehensive comparison plots for all models
    """
    # Metrics comparison
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Baseline Models Comparison', fontsize=16, fontweight='bold')

    metrics = ['Test_AUC', 'Test_F1', 'CV_AUC_Mean', 'Time_Seconds']
    titles = ['Test AUC Score', 'Test F1-Score', 'Cross-Validation AUC', 'Training Time (seconds)']

    for idx, (metric, title) in enumerate(zip(metrics, titles)):
        ax = axes[idx//2, idx%2]
        bars = ax.bar(results_df['Model'], results_df[metric], color=plt.cm.Set3(np.arange(len(results_df))))
        ax.set_title(title, fontweight='bold')
        ax.set_ylabel(metric.replace('_', ' '))
        ax.tick_params(axis='x', rotation=45)

        # Add value labels on bars
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.4f}' if metric != 'Time_Seconds' else f'{height:.1f}s',
                   ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    plt.savefig("../reports/figures/baseline_models_comparison.svg",
                format='svg', bbox_inches='tight')
    plt.close()

    # Detailed metrics plot
    plt.figure(figsize=(12, 8))
    metrics_plot = ['Test_AUC', 'Test_F1', 'Test_AP', 'CV_AUC_Mean']
    x_pos = np.arange(len(results_df))

    for i, metric in enumerate(metrics_plot):
        plt.bar(x_pos + i*0.2, results_df[metric], width=0.2,
               label=metric.replace('_', ' '))

    plt.xlabel('Models')
    plt.ylabel('Scores')
    plt.title('Detailed Model Performance Comparison', fontweight='bold')
    plt.xticks(x_pos + 0.3, results_df['Model'], rotation=45)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig("../reports/figures/detailed_performance_comparison.svg",
                format='svg', bbox_inches='tight')
    plt.close()

# Load processed data
print("Loading processed data...")
X_train = np.load("../data/processed/X_train_scaled.npy")
X_test = np.load("../data/processed/X_test_scaled.npy")
y_train = np.load("../data/processed/y_train.npy")
y_test = np.load("../data/processed/y_test.npy")

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Positive class ratio: {y_train.mean():.3f}")

# Calculate class weights
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))
print(f"Class weights: {class_weight_dict}")

# Initialize results storage
all_results = []
trained_models = {}

# Logistic Regression
print("\n" + "="*80)
print("1. LOGISTIC REGRESSION BASELINE")
print("="*80)
lr_model = LogisticRegression(
    class_weight='balanced',
    random_state=42,
    max_iter=1000,
    solver='liblinear',
    C=1.0
)
lr_results, lr_trained = evaluate_model(lr_model, X_train, y_train, X_test, y_test, "Logistic Regression")
all_results.append(lr_results)
trained_models['Logistic Regression'] = lr_trained

# Random Forest
print("\n" + "="*80)
print("2. RANDOM FOREST BASELINE")
print("="*80)
rf_model = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    max_depth=10,
    min_samples_split=5
)
rf_results, rf_trained = evaluate_model(rf_model, X_train, y_train, X_test, y_test, "Random Forest")
all_results.append(rf_results)
trained_models['Random Forest'] = rf_trained

# XGBoost
print("\n" + "="*80)
print("3. XGBOOST BASELINE")
print("="*80)
scale_pos_weight = sum(y_train == 0) / sum(y_train == 1)
xgb_model = XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6
)
xgb_results, xgb_trained = evaluate_model(xgb_model, X_train, y_train, X_test, y_test, "XGBoost")
all_results.append(xgb_results)
trained_models['XGBoost'] = xgb_trained

# LightGBM
print("\n" + "="*80)
print("4. LIGHTGBM BASELINE")
print("="*80)
lgbm_model = LGBMClassifier(
    class_weight='balanced',
    random_state=42,
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    n_jobs=-1
)
lgbm_results, lgbm_trained = evaluate_model(lgbm_model, X_train, y_train, X_test, y_test, "LightGBM")
all_results.append(lgbm_results)
trained_models['LightGBM'] = lgbm_trained

# Compare results
print("\n" + "="*80)
print("MODEL COMPARISON SUMMARY")
print("="*80)
results_df = pd.DataFrame(all_results)
print("\nDetailed Results:")
print(results_df.to_string(index=False))

# Sort by Test AUC to find best model
results_df_sorted = results_df.sort_values('Test_AUC', ascending=False)
best_model_name = results_df_sorted.iloc[0]['Model']
best_model = trained_models[best_model_name]

print(f"\n Best Model: {best_model_name} (Test AUC: {results_df_sorted.iloc[0]['Test_AUC']:.4f})")

# Generate comparison plots
plot_comparison_results(results_df)

# Save all models and results
print("\n" + "="*80)
print("SAVING MODELS AND RESULTS")
print("="*80)

# Save all models
for model_name, model in trained_models.items():
    filename = f"../models/saved_models/baseline_{model_name.replace(' ', '_').lower()}.pkl"
    joblib.dump(model, filename)
    print(f"Saved {model_name} to {filename}")

# Save best model separately
joblib.dump(best_model, "../models/saved_models/best_baseline_model.pkl")
print(f"Best model ({best_model_name}) saved separately")

# Save results to CSV
results_df.to_csv("../models/saved_models/baseline_model_results.csv", index=False)
print("Results saved to CSV")

# Save detailed comparison report
with open("../models/saved_models/baseline_models_report.txt", "w") as f:
    f.write("BASELINE MODELS COMPARISON REPORT\n")
    f.write("=" * 50 + "\n")
    f.write(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Dataset: {X_train.shape[0]:,} training samples, {X_test.shape[0]:,} test samples\n")
    f.write(f"Positive class ratio: {y_train.mean():.3f}\n\n")

    f.write("PERFORMANCE SUMMARY:\n")
    f.write("-" * 30 + "\n")
    for _, row in results_df_sorted.iterrows():
        f.write(f"{row['Model']}:\n")
        f.write(f"  Test AUC: {row['Test_AUC']:.4f}\n")
        f.write(f"  Test F1: {row['Test_F1']:.4f}\n")
        f.write(f"  CV AUC: {row['CV_AUC_Mean']:.4f} (±{row['CV_AUC_Std']:.4f})\n")
        f.write(f"  Time: {row['Time_Seconds']:.1f}s\n\n")

print("Detailed report generated")

print("\n" + "="*80)
print("BASELINE MODELING COMPLETED SUCCESSFULLY!")
print("="*80)
print(f"Best performing model: {best_model_name}")
print(f"Best Test AUC: {results_df_sorted.iloc[0]['Test_AUC']:.4f}")
print(f"Visualization files saved in '../reports/figures/'")
print(f"Models saved in '../models/saved_models/'")

Loading processed data...
Training set shape: (183824, 25)
Test set shape: (45957, 25)
Positive class ratio: 0.103
Class weights: {0: np.float64(0.5575492872308159), 1: np.float64(4.84410245599241)}

1. LOGISTIC REGRESSION BASELINE

Evaluating Logistic Regression
Training AUC: 0.8358
Test AUC: 0.8355
Test F1-Score: 0.3813
Average Precision: 0.3725
CV AUC: 0.8356 (±0.0026)
CV F1: 0.3835 (±0.0024)
Time taken: 4.55 seconds

2. RANDOM FOREST BASELINE

Evaluating Random Forest
Training AUC: 0.8570
Test AUC: 0.8327
Test F1-Score: 0.3780
Average Precision: 0.3598
CV AUC: 0.8321 (±0.0033)
CV F1: 0.3827 (±0.0037)
Time taken: 12.39 seconds

3. XGBOOST BASELINE

Evaluating XGBoost
Training AUC: 0.8563
Test AUC: 0.8372
Test F1-Score: 0.3774
Average Precision: 0.3732
CV AUC: 0.8362 (±0.0024)
CV F1: 0.3802 (±0.0011)
Time taken: 285.58 seconds

4. LIGHTGBM BASELINE

Evaluating LightGBM
[LightGBM] [Info] Number of positive: 18974, number of negative: 164850
[LightGBM] [Info] Auto-choosing row-wise mul